# s5fs (Базовая файловая система System V)

Файловая система s5fs занимает раздел диска и состоит из трех основных компонентов.
- Суперблок. Содержит общую информацию о файловой системе, например, о ее архитектуре, общем числе блоков и индексных дескрипторов, или метаданных (inode).
- Массив индексных дескрипторов (ilist). Содержит метаданные всех файлов файловой системы. Индексный дескриптор содержит информацию о состоянии файла и указывает на расположение данных этого файла. Ядро обращается к индексному дескриптору по индексу в массиве ilist. Один дескриптор является главным (корневым, root) индексным дескриптором файловой системы, через него обеспечивается доступ к структуре каталогов и файлов после монтирования файловой системы. Размер массива индексных дескрипторов фиксирован и задается при создании файловой системы. Таким образом, файловая система s5fs имеет ограни чение по числу файлов, которые могут храниться в ней, независимо от размера этих файлов (65 535 файлов в одном разделе).
- Блокы хранения данных. Данные обычных файлов и каталогов хранятся в блоках. Обработка файла осуществляется через индексный дескриптор, содержащий ссылки на блоки данных. Блоки хранения данных занимают большую часть дискового разделами их число определяет максимальный суммарный объем файлов данной файловой системы. Размер блока кратен 512 байтам, а например, файловая система S51K SCO UNIX использует размер блока в 1 Кбайт (отсюда и название).

![](./images/s5fs.png)

Суперблок считывается в память при монтировании файловой системы и находится там до ее отключения (размонтирования).
Суперблок содержит следующую информацию:
- тип файловой системы (s_type)
- размер файловой системы в логических блоках, включая сам суперблок, массив индексных дескрипторов ilist и блоки хранения данных (s_fsize)
- размер массива индексных дескрипторов (s_isize)
- число свободных блоков, доступных для размещения (s_tfree)
- число свободных индексных дескрипторов, доступных для размещение (s_tinode)
- флаги (флаг модификации s_fmod, флаг режима монтирования s_froniy)
- размер логического блока (512, 1024 или 2048 байт)
- список номеров свободных индексных дескрипторов
- список адресов свободных блоков

Поскольку число свободных индексных дескрипторов и блоков хранения данных может быть значительным, хранение двух последних списков целиком в суперблоке непрактично. Например, для индексных дескрипторов хранится только часть списка. Когда число свободных дескрипторов в этом списке приближается к 0, ядро просматривает массив ilist и вновь формирует список свободных дескрипторов inode. Для этого ядро анализирует поле di_mode индексного дескриптора, которое равно 0 у свободных inode.

К сожалению, такой подход неприменим в отношении свободных блоков хранения данных, поскольку по содержимому блока нельзя определить, свободен он или нет. Поэтому необходимо хранить список адресов свободных блоков целиком. Список адресов свободных блоков может занимать несколько блоков хранения данных, но суперблок содержит только один блок
этого списка. Первый элемент этого блока указывает на блок, хранящий продолжение списка и так далее, как это показано.

Индексный дескриптор или inode содержит информацию о файле, необходимую для обработки данных, то есть метаданные файла. Каждый файл ассоциирован с одним индексным дескриптором, хотя может иметь несколько имен в файловой системе, каждое из которых указывает на один и тот же дескриптор.
Индексный дескриптор не содержит:
- имени файла, которое находится в блоках хранения данных каталога
- содержимого файла, которое размещено в блоках хранения данных

![](./images/inode.png)


# FFS (BSD UNIX)

Организация файловой системы предусматривает логическое деление дискового раздела на одну или несколько групп цилиндров.

![](./images/ffs.png)

В отличие от s5fs, файловая система FFS при размещении блоков использует стратегию, направленную на увеличение производительности.

- Файл по возможности размещается в блоках хранения данных, принадлежащих одной группе цилиндров, где расположены его метаданные. Поскольку многие операции файловой системы включают работу, связанную как с метаданными, так и с данными файла, это правило уменьшает время совершения таких операций.
- Все файлы каталога по возможности размещаются в одной группе цилиндров. Поскольку многие команды работают с несколькими файлами одного и того же каталога, данный подход увеличивает скорость последовательного доступа к этим файлам.
-  Каждый новый каталог по возможности помещается в группу цилиндров, отличную от группы родительского каталога. Таким образом достигается равномерное распределение данных по диску.
- Последовательные блоки размещаются исходя из оптимизации физического доступа. Дело в том, что существует определенный промежуток времени между моментом завершения чтения блока и началом чтения следующего. За это время диск успеет совершить оборот на некоторые угол. Таким образом, следующий блок должен по возможности рас полагаться с пропуском нескольких секторов. В этом случае при чтении последовательных блоков не потребуется совершать "холостые" обороты диска.

# FAT

Логический раздел, отформатированный под файловую систему FAT, состоит из следующих областей.

- Загрузочный сектор содержит программу начальной загрузки операционной системы. Вид этой программы зависит от типа операционной системы, которая будет загружаться из этого раздела.
- Основная копия FAТ содержит информацию о размещении файлов и каталогов на диске.
- Резервная копия FAT.
- Корневой каталог занимает фиксированную область размером в 32 сектора (16 Кбайт), что позволяет хранить 512 записей о файлах и каталогах, так как каждая запись каталога состоит из 32 байт.
- Область данных предназначена для размещения всех файлов и всех каталогов, кроме корневого каталога.

![](./images/fat.jpg)

Индексный указатель может принимать следующие значения, характеризующие состояние связанного с ним кластера:

- кластер свободен (не используется);
- кластер используется файлом и не является последним кластером файла; в этом случае индексный указатель содержит номер следующего кластера файла;
- последний кластер файла; О дефектный кластер; а резервный кластер.